In [ ]:
import time
import datetime as dt
import pandas as pd
import pandas_ta as ta
import yfinance as yf
import matplotlib.pyplot as plt
import os
pd.options.mode.chained_assignment = None

In [ ]:
tickersdf = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

In [ ]:
tickers =  [i.replace('.','-') for i in tickersdf.Symbol.to_list()]
# Remove offending symbols as neede
tickers.remove('WRK')

In [ ]:
df = pd.read_csv('datasets/AAPL.csv.gz', index_col='Date', parse_dates=True)
df['date'] = df.index
buy, sell = getSignals(df)

frame = df.copy()
plt.figure(figsize=(20,5))
plt.scatter(frame.loc[buy].index, frame.loc[buy]['open'], marker='^',c='g')
plt.scatter(frame.loc[sell].index, frame.loc[sell]['open'], marker='*',c='r')
plt.plot(frame['close'], alpha=0.7)

In [ ]:
def myRSI(symbol):
    df['Upmove'] = df['price_change'].apply(lambda x: x if x > 0 else 0)
    df['avg_down'] = df.Downmove.ewm(span=19).mean()

def flatten(t):
    return [item for sublist in t for item in sublist]

def load_symbol(symbol):
    file = f"datasets/{symbol}.csv.gz"
    if os.path.isfile(file):
        df = pd.read_csv(file, index_col='Date', parse_dates=True)
    else:
        period = '3mo'
        df = yf.download(symbol, progress=False, start='2005-01-01')
        df.columns = df.columns.str.lower()
        df.to_csv(file,index=True)
    df['symbol'] = symbol
    df.name = symbol
    return df

def uberdf_load_all():
    start = time.time()
    frames = {}
    for filename in os.listdir('datasets'):
        symbol = filename.split(".")[0]
        df = load_symbol(symbol)
        if df.empty: continue
        frames[symbol] = df
    end = time.time()
    print(f'Elapsed: {end - start:0.2f}')
    return frames

def uberdf_one_config(udf, params):
    print(f'uberdf_one_config: {params}')
    start = time.time()
    if len(udf) == 0:
        print(f'Loading history from filesystem')
        udf = {}
        for filename in os.listdir('datasets'):
            symbol = filename.split(".")[0]
            udf[symbol] = load_symbol(symbol)
    
    frames = []
    i = 0
    for symbol in udf.keys():
        i += 1
        #if i > 2: break
        df = udf[symbol]
        #print(f'uberdf_main {i:04d}:{symbol:6s} len={len(df)}')
        df = rsi_momo_strategy(symbol, df, params)
        if i % 5 != 0: continue
        #    print(f'backtest {i:04d}:{symbol:6s} buys:{df.bprice.count()}  mean:{df.pct_return.mean():05.2f}')
        frames.append(df)

    xdf = pd.concat(frames)
    end = time.time()
    params['elapsed'] = end - start
    params['trades'] = len(xdf)
    if len(xdf) > 0:        
        params['wins'] = wins = xdf.loc[xdf.pct_return > 0].pct_return.count()
        params['pct_wins'] = params['wins'] / len(xdf)
    else:
        params['wins'] = 0
        params['pct_wins'] = 0
    params['mean'] = xdf.pct_return.mean() 
    params['std'] = xdf.pct_return.std()
    params['sum'] = xdf.pct_return.sum()
    print(f'uberdf_one_config: results={params}')
    #print(f'uberdf_one_config: mean={params["mean"]:04.2f} std={params["sum"]:04.2f} sum={params["std"]:04.2f}')
    #print(f'Summary: wins={wins} total={total} rate={wins/total*100:04.2f} pct_return={xdf.pct_return.mean()*100:05.2f}%')
    #print(xdf.pct_return.describe())
    return xdf

def uberdf_main(shard_id=None,shard_max=None):
    start = time.time()
    configs = enumerate_params()
    i = 0
    for params in configs:
        i += 1
        params['id'] = i
        if shard_id and i % shard_max != shard_id: continue
        uberdf_one_config(udf, params)
    end = time.time()
    print(f'uberdf_main: Elapsed: {end - start:0.2f}')
    return configs

udf = uberdf_load_all()
results = uberdf_main()


In [116]:
def enumerate_params():
    keys = ['rsi_entry', 'rsi_exit', 'sma_period', 'stop_loss_pct', 'max_lookahead']
    params =  []
    for rsi_entry in range(15,31,5):
        for rsi_exit in range(35,46,5):
            for sma_period in range(100,201,20):
                for stop_loss_pct in range(0,6,1):
                    for max_lookahead in range(4,17,2):
                        param = dict(zip(keys, [rsi_entry, rsi_exit, sma_period, stop_loss_pct, max_lookahead]))
                        params.append(param)
    return params

def enumerate_params_products():
    keys = ['rsi_entry', 'rsi_exit', 'sma_period', 'stop_loss_pct', 'max_lookahead']
    params =  []
    r_rsi_entry = list(range(15,31,5))
    r_rsi_exit = list(range(35,46,5))
    r_sma_period = list(range(100,201,20))
    r_stop_loss_pct = list(range(0,6,1))
    r_max_lookahead = list(range(4,17,2))
    list(itertools.product(r_rsi_entry,r_rsi_exit, r_sma_period, r_stop_loss_pct, r_max_lookahead))
    return params

def in_trade(idx, trades):
    for t in trades:
        if t[1] <= idx and t[3] > idx: return True
    return False

def rsi_momo_strategy(symbol, df, params):
    stop_loss_pct = params['stop_loss_pct']
    df = df.copy()
    df.name = symbol
    df['rsi'] = df.ta.rsi()
    df['sma'] = df.ta.sma(params['sma_period'])
    df.dropna(inplace=True)
    
    cutoff = df.tail(params['max_lookahead']+1).index.min()
    
    # RSI xUnder
    signals = df.loc[(df.close > df.sma) & (df.rsi < params['rsi_entry']) & (df.index < cutoff)]
    # RSI xOver
    signals = df.loc[(df.close > df.sma) & (df.rsi > params['rsi_entry']) & (df.shift(1).rsi < params['rsi_entry']) & (df.index < cutoff)]
    trades = []

    def apply_trade(row):
        if in_trade(row.name, trades): return None
        iloc = df.index.get_loc(row.name)
        buy_row = df.iloc[iloc+1]
        sell_stop = None
        if params['stop_loss_pct']:
            sell_stop = buy_row.open * (1.0 - (params['stop_loss_pct']/100))
        sell_row = None
        sell_descr = 'max_days'
        for j in range(1,params['max_lookahead']):
            this_day = df.iloc[iloc + j]
            sell_row = df.iloc[iloc + j + 1]
            if this_day.rsi > params['rsi_exit']:
                sell_descr = 'xOverRSI'
                break
            if sell_stop and this_day.low < sell_stop:
                sell_descr = 'stop_loss'
                break
        pct_return = sell_row.open / buy_row.open - 1
        trades.append([symbol, buy_row.name, buy_row.open, sell_row.name, sell_row.open, pct_return, sell_descr])
        return None

    signals.apply(lambda row: apply_trade(row), axis=1)
    return pd.DataFrame(trades, columns=['symbol', 'bdate', 'bprice', 'sdate', 'sprice', 'pct_return', 'sell_descr'])

# Testing section
params={'rsi_entry': 30, 'rsi_exit': 40, 'sma_period': 200, 'stop_loss_pct': 0, 'max_lookahead': 16}
symbol = 'AAPL'
df = load_symbol(symbol)
trades = rsi_momo_strategy(symbol, df, params)
print(f'rsi_momo_strategy\n{trades}')


rsi_momo_strategy
  symbol      bdate      bprice      sdate      sprice  pct_return sell_descr
0   AAPL 2006-03-29    2.111786 2006-03-30    2.243571    0.062405   xOverRSI
1   AAPL 2015-07-10   30.485001 2015-07-13   31.257500    0.025340   xOverRSI
2   AAPL 2016-11-07   27.520000 2016-11-09   27.469999   -0.001817   xOverRSI
3   AAPL 2016-11-15   26.642500 2016-11-17   27.452499    0.030403   xOverRSI
4   AAPL 2018-02-05   39.775002 2018-02-13   40.487499    0.017913   xOverRSI
5   AAPL 2020-02-28   64.315002 2020-03-03   75.917503    0.180401   xOverRSI
6   AAPL 2021-02-26  122.589996 2021-03-02  128.410004    0.047475   xOverRSI


In [ ]:
df = pd.DataFrame(results)
df.to_csv('./rsi_momo.results.csv')
params={'rsi_entry': 30, 'rsi_exit': 40, 'sma_period': 200, 'stop_loss_pct': 0, 'max_lookahead': 16}
df = uberdf_one_config(udf, params)
df.to_csv('./rsi_momo.best-params-trades.csv')
df

In [117]:
df

,open,high,low,close,adj close,volume,symbol
Date,,,,,,,
2005-01-03,1.156786,1.162679,1.117857,1.130179,0.971844,691992000,AAPL
2005-01-04,1.139107,1.169107,1.124464,1.141786,0.981825,1096810400,AAPL
2005-01-05,1.151071,1.165179,1.143750,1.151786,0.990424,680433600,AAPL
2005-01-06,1.154821,1.159107,1.130893,1.152679,0.991192,705555200,AAPL
2005-01-07,1.160714,1.243393,1.156250,1.236607,1.063362,2227450400,AAPL
...,...,...,...,...,...,...,...
2021-07-19,143.750000,144.070007,141.669998,142.449997,142.449997,121434600,AAPL
2021-07-20,143.460007,147.100006,142.960007,146.149994,146.149994,96238600,AAPL
2021-07-21,145.529999,146.130005,144.630005,145.399994,145.399994,74915000,AAPL
